In [6]:
#!/usr/bin/env python3
import os
import json
import cv2
from tqdm import tqdm
from collections import defaultdict

# =====================================================
# ✅ 版本配置
# =====================================================
VERSION = "0.5"
SCORE_THRESHOLD = 0.7  # 仅用于信息展示（已在 filter 阶段完成）
BASE_DIR = "/opt/data/private/BlackBox"

# 数据与路径配置
PATCH_IMG_DIR = f"{BASE_DIR}/data/coco-patch-{VERSION}/val2017/"
RESULT_BASE = f"{BASE_DIR}/save-{VERSION}/attack/detection"
RES_SAVE_BASE = f"{BASE_DIR}/save-{VERSION}/attack/res"

MODEL_NAMES = ["detr", "deformable-detr", "sparse-detr", "anchor-detr", "dn-detr"]

# =====================================================
# 工具函数
# =====================================================
def load_json(path):
    if not os.path.exists(path):
        print(f"⚠️ 文件不存在: {path}")
        return []
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def draw_boxes(image, detections, color=(0, 0, 255)):
    """绘制检测框"""
    img = image.copy()
    for det in detections:
        bbox = det.get("bbox", [])
        if len(bbox) != 4:
            continue
        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
    return img

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)
    return path

# =====================================================
# 核心逻辑
# =====================================================
def process_model(model_name):
    print(f"\n=== 🚀 {model_name.upper()} 模型 Recall 计算开始 ===")

    # 文件路径
    model_dir = os.path.join(RESULT_BASE, model_name, "patch")
    clean_path = os.path.join(model_dir, "clean-filter.json")
    patch_path = os.path.join(model_dir, "res-std-filter.json")

    # 输出路径
    report_dir = ensure_dir(os.path.join(RES_SAVE_BASE, model_name))
    visual_dir = ensure_dir(os.path.join(report_dir, "visual"))
    success_dir = ensure_dir(os.path.join(report_dir, "success"))
    report_path = os.path.join(report_dir, "recall_attack_report.json")

    # 加载检测结果
    clean_data = load_json(clean_path)
    patch_data = load_json(patch_path)

    if not clean_data or not patch_data:
        print(f"⚠️ {model_name}: 缺少 clean 或 patch 检测数据，跳过。")
        return

    # 统计 clean / patch
    clean_group = defaultdict(list)
    patch_group = defaultdict(list)
    for det in clean_data:
        clean_group[det["image_id"]].append(det)
    for det in patch_data:
        patch_group[det["image_id"]].append(det)

    total_clean = len(clean_data)
    total_patched = len(patch_data)
    recall = round(total_patched / total_clean, 4) if total_clean > 0 else 0.0

    # 计算成功图片（基于 image_id）
    success_imgs = []
    success_details = []
    for img_id, clean_dets in clean_group.items():
        clean_cnt = len(clean_dets)
        patch_cnt = len(patch_group.get(img_id, []))
        if clean_cnt == 0:
            continue
        if patch_cnt < clean_cnt:  # 攻击成功
            status = "full" if patch_cnt == 0 else "partial"
            success_imgs.append(img_id)
            success_details.append({
                "image_id": img_id,
                "file_name": clean_dets[0]["file_name"],
                "status": status,
                "clean_count": clean_cnt,
                "patched_count": patch_cnt,
                "reduction": clean_cnt - patch_cnt
            })

    # 🎨 绘制并保存可视化图片
    print("🎨 开始绘制与保存可视化结果...")
    for img_id, dets in tqdm(patch_group.items(), desc=f"{model_name} 可视化中"):
        file_name = dets[0]["file_name"]
        img_path = os.path.join(PATCH_IMG_DIR, file_name)
        if not os.path.exists(img_path):
            continue
        img = cv2.imread(img_path)
        vis_img = draw_boxes(img, dets, color=(0, 0, 255))
        cv2.imwrite(os.path.join(visual_dir, f"{img_id}_{file_name}"), vis_img)

    # 💥 保存成功图片
    print("💥 提取攻击成功图片...")
    for info in success_details:
        img_path = os.path.join(PATCH_IMG_DIR, info["file_name"])
        if not os.path.exists(img_path):
            continue
        img = cv2.imread(img_path)
        dets = patch_group.get(info["image_id"], [])
        if dets:  # 部分成功，有检测框
            img = draw_boxes(img, dets, color=(0, 255, 255))
        save_path = os.path.join(success_dir, f"{info['image_id']}_{info['status']}_{info['file_name']}")
        cv2.imwrite(save_path, img)

    # 📊 生成报告
    report = {
        "model": model_name,
        "version": VERSION,
        "core_metrics": {
            "clean_person_total": total_clean,
            "patched_person_total": total_patched,
            "recall": recall,
            "recall_percent": f"{recall * 100:.2f}%",
            "total_reduction": total_clean - total_patched
        },
        "successful_images": {
            "count": len(success_imgs),
            "details": success_details
        },
        "paths": {
            "clean_json": clean_path,
            "patch_json": patch_path,
            "visual_dir": visual_dir,
            "success_dir": success_dir
        }
    }

    # 保存报告
    with open(report_path, "w", encoding="utf-8") as f:
        json.dump(report, f, ensure_ascii=False, indent=2)

    print(f"✅ {model_name.upper()} 报告完成: Recall={recall*100:.2f}% | 成功图片: {len(success_imgs)} 张")
    print(f"📄 报告路径: {report_path}\n")


# =====================================================
# 主函数
# =====================================================
def main():
    print(f"=== Patch vs Clean Recall 计算开始 (VERSION={VERSION}) ===")
    for model_name in MODEL_NAMES:
        process_model(model_name)
    print("\n✅ 所有模型 Recall 计算与成功图片保存完成。")


if __name__ == "__main__":
    main()


=== Patch vs Clean Recall 计算开始 (VERSION=0.5) ===

=== 🚀 DETR 模型 Recall 计算开始 ===
🎨 开始绘制与保存可视化结果...


detr 可视化中: 100%|██████████| 287/287 [00:10<00:00, 27.44it/s]


💥 提取攻击成功图片...
✅ DETR 报告完成: Recall=97.83% | 成功图片: 17 张
📄 报告路径: /opt/data/private/BlackBox/save-0.5/attack/res/detr/recall_attack_report.json


=== 🚀 DEFORMABLE-DETR 模型 Recall 计算开始 ===
🎨 开始绘制与保存可视化结果...


deformable-detr 可视化中: 100%|██████████| 233/233 [00:08<00:00, 28.15it/s]


💥 提取攻击成功图片...
✅ DEFORMABLE-DETR 报告完成: Recall=71.61% | 成功图片: 108 张
📄 报告路径: /opt/data/private/BlackBox/save-0.5/attack/res/deformable-detr/recall_attack_report.json


=== 🚀 SPARSE-DETR 模型 Recall 计算开始 ===
🎨 开始绘制与保存可视化结果...


...

💥 提取攻击成功图片...
✅ SPARSE-DETR 报告完成: Recall=90.35% | 成功图片: 41 张
📄 报告路径: /opt/data/private/BlackBox/save-0.5/attack/res/sparse-detr/recall_attack_report.json


=== 🚀 ANCHOR-DETR 模型 Recall 计算开始 ===
🎨 开始绘制与保存可视化结果...


anchor-detr 可视化中: 100%|██████████| 236/236 [00:08<00:00, 28.02it/s]


💥 提取攻击成功图片...
✅ ANCHOR-DETR 报告完成: Recall=84.84% | 成功图片: 55 张
📄 报告路径: /opt/data/private/BlackBox/save-0.5/attack/res/anchor-detr/recall_attack_report.json


=== 🚀 DN-DETR 模型 Recall 计算开始 ===
🎨 开始绘制与保存可视化结果...


dn-detr 可视化中: 100%|██████████| 232/232 [00:07<00:00, 29.66it/s]


💥 提取攻击成功图片...
✅ DN-DETR 报告完成: Recall=88.01% | 成功图片: 50 张
📄 报告路径: /opt/data/private/BlackBox/save-0.5/attack/res/dn-detr/recall_attack_report.json


✅ 所有模型 Recall 计算与成功图片保存完成。


# 数据整合成一个表格

In [12]:
#!/usr/bin/env python3
import os
import json
import pandas as pd
import matplotlib.pyplot as plt

# =====================================================
# ✅ 配置
# =====================================================
VERSION = "0.5"
BASE_DIR = "/opt/data/private/BlackBox"
RES_BASE = f"{BASE_DIR}/save-{VERSION}/attack/res"

# 固定模型顺序（显示顺序）
MODEL_NAMES = ["detr", "deformable-detr", "sparse-detr", "anchor-detr", "dn-detr"]

# =====================================================
# ✅ 加载单模型报告
# =====================================================
def load_report(model_name):
    """读取单模型 recall_attack_report.json"""
    path = os.path.join(RES_BASE, model_name, "recall_attack_report.json")
    if not os.path.exists(path):
        print(f"⚠️ 未找到报告: {path}")
        return None
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    core = data.get("core_metrics", {})
    succ = data.get("successful_images", {})
    return {
        "Model": model_name,
        "Version": data.get("version", "-"),
        "Clean Boxes": core.get("clean_person_total", 0),
        "Patched Boxes": core.get("patched_person_total", 0),
        "Reduction": core.get("total_reduction", 0),
        "Recall": f"{core.get('recall', 0.0):.4f}",
        "Recall (%)": f"{float(core.get('recall', 0.0)) * 100:.2f}",
        "Success Images": succ.get("count", 0)
    }

# =====================================================
# ✅ 保存为自适应列宽 PNG 表格
# =====================================================
def save_table_as_png(df, output_path, title="Model Recall Comparison (Fixed Order)"):
    """保存干净自适应宽度表格为 PNG"""
    # 动态列宽计算
    col_widths = []
    for col in df.columns:
        max_len = max(df[col].astype(str).map(len).max(), len(col))
        col_widths.append(max_len * 0.12)  # 每个字符约0.12英寸宽

    total_width = sum(col_widths)
    fig_height = 1.2 + len(df) * 0.4
    fig, ax = plt.subplots(figsize=(total_width, fig_height))
    ax.axis("off")

    table = ax.table(
        cellText=df.values,
        colLabels=df.columns,
        loc="center",
        cellLoc="center",
        rowLoc="center"
    )

    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 1.2)

    # 设置列宽比例
    for i, width in enumerate(col_widths):
        for j in range(len(df) + 1):  # 包括表头
            cell = table[(j, i)]
            cell.set_width(width / total_width)

    # 白底 + 黑线边框
    for _, cell in table.get_celld().items():
        cell.set_edgecolor("black")
        cell.set_linewidth(0.6)
        cell.set_facecolor("white")

    plt.title(title, fontsize=11, pad=10)
    plt.savefig(output_path, dpi=300, bbox_inches="tight", pad_inches=0.1)
    plt.close(fig)
    print(f"✅ 已保存自适应列宽表格图片: {output_path}")

# =====================================================
# ✅ 主逻辑
# =====================================================
def main():
    print(f"=== 📊 汇总 Recall 结果并输出 PNG 表格 (VERSION={VERSION}) ===")

    records = []
    for model in MODEL_NAMES:
        info = load_report(model)
        if info:
            records.append(info)

    if not records:
        print("⚠️ 未找到任何模型报告。")
        return

    df = pd.DataFrame(records)
    df = df[["Model", "Version", "Clean Boxes", "Patched Boxes", "Reduction",
             "Recall", "Recall (%)", "Success Images"]]

    # ✅ 固定模型顺序（不按Recall排序）
    df["Model"] = pd.Categorical(df["Model"], categories=MODEL_NAMES, ordered=True)
    df = df.sort_values("Model").reset_index(drop=True)

    # 输出路径
    csv_path = os.path.join(RES_BASE, "recall_summary.csv")
    md_path = os.path.join(RES_BASE, "recall_summary.md")
    png_path = os.path.join(RES_BASE, "recall_summary.png")

    # 保存 CSV & Markdown
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    with open(md_path, "w", encoding="utf-8") as f:
        f.write(df.to_markdown(index=False))

    # 保存 PNG（自动适应内容宽度）
    save_table_as_png(df, png_path, title=f"Model Recall Comparison (Version {VERSION})")

    print(f"\n✅ 已保存 CSV: {csv_path}")
    print(f"✅ 已保存 Markdown: {md_path}")
    print(f"✅ 已保存 PNG: {png_path}\n")

    print("=== ✅ 汇总完成 ===\n")
    print(df.to_string(index=False))


if __name__ == "__main__":
    main()


=== 📊 汇总 Recall 结果并输出 PNG 表格 (VERSION=0.5) ===
✅ 已保存自适应列宽表格图片: /opt/data/private/BlackBox/save-0.5/attack/res/recall_summary.png

✅ 已保存 CSV: /opt/data/private/BlackBox/save-0.5/attack/res/recall_summary.csv
✅ 已保存 Markdown: /opt/data/private/BlackBox/save-0.5/attack/res/recall_summary.md
✅ 已保存 PNG: /opt/data/private/BlackBox/save-0.5/attack/res/recall_summary.png

=== ✅ 汇总完成 ===

          Model Version  Clean Boxes  Patched Boxes  Reduction Recall Recall (%)  Success Images
           detr     0.5          599            586         13 0.9783      97.83              17
deformable-detr     0.5          539            386        153 0.7161      71.61             108
    sparse-detr     0.5          549            496         53 0.9035      90.35              41
    anchor-detr     0.5          488            414         74 0.8484      84.84              55
        dn-detr     0.5          492            433         59 0.8801      88.01              50
